# Moisture Magic 💦 🧙‍♂️
## Preprocessing of UFZ Dataset Soil Moisture "Hohes Holz" 📝 → 📦

HIDA 2020 Hackthon, 2020-11-05

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# read raw data and convert datetime
df = pd.read_csv("/glacier2/dldatasets/hida2020-ufz-data/00_data.csv", delim_whitespace=True)
df['Date.Time'] = pd.to_datetime(df['Date.Time'])
df.head(2)

,,Time,Box,Date.Time,SM1,SM1_Flag,Temp1,Temp1_Flag,SM2,SM2_Flag,Temp2,...,SM6_Flag,Temp6,Temp6_Flag,BattV,SM1_Depth,SM2_Depth,SM3_Depth,SM4_Depth,SM5_Depth,SM6_Depth
1.2,2012-07-19,17:45:53,1,2012-07-19 17:45:53,-2.8240,Auto:Range,18.2580,OK,-5.9817,Auto:Range,18.2232,...,Auto:Range,18.3623,OK,3599,0.05,0.05,0.05,0.05,0.05,0.05
2.2,2010-10-03,02:07:00,2,2010-10-03 02:07:00,29.9656,OK,5.0416,OK,-4.8066,Auto:Range,-15.1656,...,OK,5.7372,OK,3138,0.10,0.20,0.30,0.40,0.50,0.60


### 1. Initial cleanup

Steps:  
* drop columns not needed for analysis
* eliminate Box1 (located in first row) since it only has one measly measurement...
* rename some columns
* make time the index

In [3]:
df = df.reset_index().drop(['level_0','level_1','Time'], axis=1).iloc[1:,:]
df = df.rename(columns={'Box': 'box', 'Date.Time': 'time'})
df = df.set_index('time')
df.head(2)

,box,SM1,SM1_Flag,Temp1,Temp1_Flag,SM2,SM2_Flag,Temp2,Temp2_Flag,SM3,...,SM6_Flag,Temp6,Temp6_Flag,BattV,SM1_Depth,SM2_Depth,SM3_Depth,SM4_Depth,SM5_Depth,SM6_Depth
time,,,,,,,,,,,,,,,,,,,,,
2010-10-03 02:07:00,2,29.9656,OK,5.0416,OK,-4.8066,Auto:Range,-15.1656,OK,-7.6653,...,OK,5.7372,OK,3138,0.1,0.2,0.3,0.4,0.5,0.6
2010-10-03 03:07:00,2,26.1448,OK,8.4848,OK,23.7972,OK,7.7544,OK,125.1174,...,Auto:Spike,6.8154,OK,3496,0.1,0.2,0.3,0.4,0.5,0.6


### 2. Resample to consistent 15min intervals

In [4]:
df = df.groupby('box').resample('15min').first().drop('box', axis=1)

### 3. Reshape dataframe (wide to short form)

Steps:
* split into variable groups
* ignore depth (we use level for vertical stratification)
* rename columns to level id for later melt
* melt to narrow df form
* convert to xarray dataset

In [5]:
# split df for variable groups
def expand_name(var, flag=False):
    f = '_Flag' if flag else ''
    return [f"{var}{i}{f}" for i in range(1,7)]

df_sm = df[expand_name('SM')]
df_temp = df[expand_name('Temp')]
df_bat = df[['BattV']]

# and quality flags
df_sm_qa = df[expand_name('SM', flag=True)]
df_temp_qa = df[expand_name('Temp', flag=True)]

# rename columns
for d in [df_sm, df_temp, df_sm_qa, df_temp_qa]:
    d.columns = list(range(1,7))

# melt
df_sm = pd.melt(df_sm.reset_index(), id_vars=['box','time']).set_index(['time','box'])
df_temp = pd.melt(df_temp.reset_index(), id_vars=['box','time']).set_index(['time','box'])
df_sm_qa = pd.melt(df_sm_qa.reset_index(), id_vars=['box','time']).set_index(['time','box'])
df_temp_qa = pd.melt(df_temp_qa.reset_index(), id_vars=['box','time']).set_index(['time','box'])

# rename new columns
df_sm.columns = ['level', 'soilmoisture']
df_temp.columns = ['level', 'soiltemp']
df_sm_qa.columns = ['level', 'soilmoisture_flag']
df_temp_qa.columns = ['level', 'soiltemp_flag']

# add level to multi-index
df_sm = df_sm.set_index('level', append=True)
df_temp = df_temp.set_index('level', append=True)
df_sm_qa = df_sm_qa.set_index('level', append=True)
df_temp_qa = df_temp_qa.set_index('level', append=True)

# convert dtype of categorical columns
df_sm_qa['soilmoisture_flag'] = df_sm_qa['soilmoisture_flag'].astype('category')
df_temp_qa['soiltemp_flag'] = df_temp_qa['soiltemp_flag'].astype('category')

df_sm_qa.head(2)

,,,soilmoisture_flag
time,box,level,
2010-10-03 02:00:00,2,1,OK
2010-10-03 02:15:00,2,1,NaN


In [6]:
display(df_sm_qa['soilmoisture_flag'].cat.categories)
display(np.unique(df_sm_qa['soilmoisture_flag'].cat.codes))

Index(['Auto:BattV', 'Auto:Range', 'Auto:Spike', 'Manual', 'OK'], dtype='object')

array([-1,  0,  1,  2,  3,  4], dtype=int8)

### 4. Convert to xarray dataset

In [7]:
ds = pd.concat([df_sm, df_temp, df_sm_qa, df_temp_qa], axis=1).to_xarray()

ds['soiltemp'].attrs['long_name'] = "soil temperature"
ds['soiltemp'].attrs['units'] = "deg C"
ds['soilmoisture'].attrs['long_name'] = "soil moisture"
ds['soilmoisture'].attrs['units'] = "vol percent"

ds['soiltemp_flag'].attrs['long_name'] = "soil temperature qa flag"
ds['soiltemp_flag'].attrs['units'] = "category"
ds['soiltemp_flag'].attrs['flag:flag_values'] = ','.join([str(x) for x in np.unique(df_sm_qa['soilmoisture_flag'].cat.codes.values)[1:]])
ds['soiltemp_flag'].attrs['flag:flag_meanings'] = ' '.join(df_temp_qa['soiltemp_flag'].cat.categories.values.tolist())
   
ds['soilmoisture_flag'].attrs['long_name'] = "soil moisture qa flag"
ds['soilmoisture_flag'].attrs['units'] = "category"

ds

<xarray.Dataset>
Dimensions:            (box: 39, level: 6, time: 302043)
Coordinates:
  * time               (time) datetime64[ns] 2010-09-30T02:00:00 ... 2019-05-...
  * box                (box) int64 2 3 4 5 6 7 8 9 ... 33 34 35 36 37 38 39 40
  * level              (level) int64 1 2 3 4 5 6
Data variables:
    soilmoisture       (time, box, level) float64 nan nan nan ... -13.3 45.43
    soiltemp           (time, box, level) float64 nan nan nan ... -15.55 7.928
    soilmoisture_flag  (time, box, level) object nan nan ... 'Manual'
    soiltemp_flag      (time, box, level) object nan nan nan ... 'Manual' 'OK'

In [8]:
df_bat = df[['BattV']]
df_bat = df_bat.reset_index()
df_bat.columns = ['box', 'time','battery']
df_bat = df_bat.set_index(['time', 'box'])
ds_bat = df_bat.to_xarray()

ds_bat.attrs['long_name'] = "battery voltage"
ds_bat.attrs['units'] = "mV"

ds_bat

<xarray.Dataset>
Dimensions:  (box: 39, time: 302043)
Coordinates:
  * time     (time) datetime64[ns] 2010-09-30T02:00:00 ... 2019-05-12T08:30:00
  * box      (box) int64 2 3 4 5 6 7 8 9 10 11 ... 31 32 33 34 35 36 37 38 39 40
Data variables:
    battery  (time, box) float64 nan nan nan ... 3.47e+03 3.248e+03 3.392e+03
Attributes:
    long_name:  battery voltage
    units:      mV

In [9]:
ds = xr.merge([ds, ds_bat])
ds

<xarray.Dataset>
Dimensions:            (box: 39, level: 6, time: 302043)
Coordinates:
  * time               (time) datetime64[ns] 2010-09-30T02:00:00 ... 2019-05-...
  * box                (box) int64 2 3 4 5 6 7 8 9 ... 33 34 35 36 37 38 39 40
  * level              (level) int64 1 2 3 4 5 6
Data variables:
    soilmoisture       (time, box, level) float64 nan nan nan ... -13.3 45.43
    soiltemp           (time, box, level) float64 nan nan nan ... -15.55 7.928
    soilmoisture_flag  (time, box, level) object nan nan ... 'Manual'
    soiltemp_flag      (time, box, level) object nan nan nan ... 'Manual' 'OK'
    battery            (time, box) float64 nan nan nan ... 3.248e+03 3.392e+03

In [10]:
ds.to_netcdf("ufz_hohes-holz.nc")

In [11]:
! ls -lh

total 2.5G
-rw-rw-r-- 1 werner-ch werner-ch  12K Nov  5 12:32 LICENSE
-rw-rw-r-- 1 werner-ch werner-ch   30 Nov  5 12:32 README.md
-rw-rw-r-- 1 werner-ch werner-ch   15 Nov  5 21:06 requirements.txt
-rw-r--r-- 1 werner-ch werner-ch  41K Nov  5 21:09 sm_data_hohesholtz_to_xr.ipynb
-rw-rw-r-- 1 werner-ch werner-ch 102K Nov  6 00:00 soilmagic_preprocessing.ipynb
-rw-rw-r-- 1 werner-ch werner-ch 2.5G Nov  6 00:00 ufz_hohes-holz.nc
